# No More Alzheimer's Disease

## Pre-Processing

Reads all of the files and attempts to merge them based on the *VISCODE* and *RID* attributes.

In [3]:
# Imports
import os
import pandas as pd

def read_all_files(directory="Data/Plasma", axis=["VISCODE", "RID"]):
    '''
        This reads all of the .csv files in the directory and outputs them. 
        
        Args : directory(String) : the path of the files to be read
        Returns : data(DataFrame) : The sum of the dataframes.'''

    # Get the name of every file in the directory
    files = os.listdir(directory)
    csv_files = list(filter(lambda f: f.endswith('.csv'), files))

    print(csv_files)

    df = pd.read_csv(directory + "/" + csv_files[0])

    # Read all of the file directories in the folder
    for i in range(1, len(csv_files)):
        temp_df = pd.read_csv(directory + "/" + csv_files[i])
        # df should be meregd with the previously constructed one
        df = pd.merge(df, temp_df, on=axis, suffixes=('_' + str(i-1), '_' + str(i)), how="outer")


        print(df)



    df.to_csv("newcsv.csv")
read_all_files()
    


['ADNI_BLENNOWPLASMANFL_10_03_18_27Nov2023.csv', 'Archive ADNI_BLENNOWPLASMANFL_27Nov2023.csv', 'FNIH_PLASMA_PTAU181_PROJECT_27Nov2023.csv', 'PLASMA_ABETA_PROJECT_ADX_VUMC_27Nov2023.csv', 'UGOTPTAU181_06_18_20_27Nov2023.csv', 'UPENNPLASMA_27Nov2023.csv']
        RID VISCODE VISCODE2_0  EXAMDATE_0 BOX_POS_0    VOL_0  PLASMA_NFL_0  \
0         2      bl         bl  2005-09-08      1-21  0.50 ML          28.2   
1         3      bl         bl  2005-09-12      1-22  0.50 ML         188.3   
2         4      bl         bl  2005-11-08      1-09  0.50 ML          31.2   
3         5      bl         bl  2005-09-07      1-20  0.50 ML          36.3   
4         6      bl         bl  2005-11-29      1-03  0.50 ML          52.5   
..      ...     ...        ...         ...       ...      ...           ...   
835  999999     NaN        NaN         NaN      1-27  0.50 ML          41.2   
836  999999     NaN        NaN         NaN      1-27  0.50 ML          41.2   
837  999999     NaN        NaN    